# Vertex Forecast - workflow orchestration with Vertex Pipelines

This notebook demonstrate the basic building blocks of using Vertex Pipelines to orchestrate Vertex Forecast modeling workflows

In [1]:
import os

GCP_PROJECTS = !gcloud config get-value project
PROJECT_ID = GCP_PROJECTS[0]
PROJECT_NUM = !gcloud projects list --filter="$PROJECT_ID" --format="value(PROJECT_NUMBER)"
PROJECT_NUM = PROJECT_NUM[0]
LOCATION = 'us-central1'
BQ_LOCATION='US'

print(f"PROJECT_ID: {PROJECT_ID}")
print(f"PROJECT_NUM: {PROJECT_NUM}")
print(f"LOCATION: {LOCATION}")

PROJECT_ID: hybrid-vertex
PROJECT_NUM: 934903580331
LOCATION: us-central1


In [64]:
from datetime import datetime
import json
import os
import time
from typing import Any, Callable, Dict, NamedTuple, Optional
import pandas as pd

import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime, timedelta

# google cloud
from google.api_core import exceptions as google_exceptions
from google_cloud_pipeline_components import aiplatform as gcc_aip
from google_cloud_pipeline_components.experimental import forecasting as gcc_aip_forecasting
import google.cloud.aiplatform as vertex_ai
from google.cloud import bigquery
from google.cloud import storage

# kfp
import kfp
import kfp.v2.dsl
from kfp.v2.google import client as pipelines_client
from kfp.v2.dsl import (
    Artifact, Dataset, 
    Input, InputPath, 
    Model, Output,
    OutputPath, component, 
    Metrics
)

### check package versions

In [5]:
print(f'kfp version: {kfp.__version__}')
! python3 -c "import google_cloud_pipeline_components; print('google_cloud_pipeline_components version: {}'.format(google_cloud_pipeline_components.__version__))"
print(f'vertex_ai SDK version: {vertex_ai.__version__}')
print(f'bigquery SDK version: {bigquery.__version__}')

kfp version: 1.8.19
google_cloud_pipeline_components version: 1.0.41
vertex_ai SDK version: 1.22.1
bigquery SDK version: 3.7.0


In [6]:
bq_client = bigquery.Client(
    project=PROJECT_ID, 
    location=BQ_LOCATION
)

storage_client = storage.Client(project=PROJECT_ID)

vertex_ai.init(
    project=PROJECT_ID,
    location=LOCATION
)

## custom pipeline components

### create BigQuery dataset

> place to put eval + forecast output

**TODO**
* explain this custom compnent e.g., NamedTuple output

In [48]:
@kfp.v2.dsl.component(
  base_image='python:3.9',
  packages_to_install=['google-cloud-bigquery==3.7.0'],
)
def create_bq_dataset(
    project: str,
    # vertex_dataset: str,
    new_bq_dataset: str,
    bq_location: str
) -> NamedTuple('Outputs', [
    ('bq_dataset_name', str),
    ('bq_dataset_uri', str),
]):
    
    from google.cloud import bigquery

    bq_client = bigquery.Client(project=project, location='US') # bq_location)
    (
      bq_client.query(f'CREATE SCHEMA IF NOT EXISTS `{project}.{new_bq_dataset}`')
      .result()
    )
    
    return (
        f'{new_bq_dataset}',
        f'bq://{project}:{new_bq_dataset}',
    )

### create input table specs

In [69]:
@kfp.v2.dsl.component(
    base_image='python:3.9',
    packages_to_install=[
        'google-cloud-aiplatform==1.22.1'
    ],
)
def create_input_table_specs(
    project: str,
    location: str,
    column_spec_gcs_bucket_name: str,
    column_spec_gcs_blob_path: str,
    bq_source_train_uri: str,
    data_granularity_unit: str,
    data_granularity_count: int,
) -> NamedTuple('Outputs', [
    ('input_table_specs', str),
    ('model_feature_columns', str),
    # ('column_specs', str),
    # ('unavailable_at_forecast_columns', str),
    # ('available_at_forecast_columns', str),
    # ('time_series_identifier_column', str),
    # ('time_column', str),
    # ('target_column', str),
    # ('predefined_split_column_name', str),
]):
    
    # here
    import os
    import json
    import pickle as pkl
    from pprint import pprint
    import logging
    logging.getLogger().setLevel(logging.INFO)
    
    # google cloud
    from google.cloud import aiplatform as vertex_ai
    from google.cloud import storage
    from google.cloud.storage.bucket import Bucket
    from google.cloud.storage.blob import Blob
    
    # set clients
    vertex_ai.init(
        project=project,
        location=location,
    )
    storage_client = storage.Client(project=project)
    
    # check vars in the logs
    logging.info(f"column_spec_gcs_bucket_name: {column_spec_gcs_bucket_name}")
    logging.info(f"column_spec_gcs_blob_path: {column_spec_gcs_blob_path}")
    
    # ===================================================
    # helper function for downloading gcs blob
    # ===================================================
    def download_blob(bucket_name, source_gcs_obj, local_filename):
        """Uploads a file to the bucket."""
        # storage_client = storage.Client(project=project_number)
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(source_gcs_obj)
        blob.download_to_filename(local_filename)
        
        filehandler = open(f'{local_filename}', 'rb')
        loaded_dict = pkl.load(filehandler)
        filehandler.close()
        
        logging.info(f"File {local_filename} downloaded from gs://{bucket_name}/{source_gcs_obj}")
        
        return loaded_dict
    
    # ===================================================
    # load pickled column specs
    # ===================================================
    
    # candidate features
    LOCAL_COL_FILE = 'lodaded_column_specs.pkl'
    logging.info(f"LOCAL_COL_FILE: {LOCAL_COL_FILE}")
    
    loaded_column_spec_dict = download_blob(
        column_spec_gcs_bucket_name,
        column_spec_gcs_blob_path,
        LOCAL_COL_FILE
    )
    
    COL_TRANSFORMS = loaded_column_spec_dict['column_specs']
    UNAVAILABLE_AT_FORECAST_COLS = loaded_column_spec_dict['unavailable_at_forecast_columns']
    AVAILABLE_AT_FORECAST_COLS = loaded_column_spec_dict['available_at_forecast_columns']
    SERIES_COLUMN = loaded_column_spec_dict['time_series_identifier_column']
    TIME_COLUMN = loaded_column_spec_dict['time_column']
    TARGET_COLUMN = loaded_column_spec_dict['target_column']
    PREDEFINED_SPLIT_COL = loaded_column_spec_dict['predefined_split_column_name']
    
    model_feature_columns = [
        PREDEFINED_SPLIT_COL,
        SERIES_COLUMN,
        TARGET_COLUMN,
        TIME_COLUMN
    ]
    for col in UNAVAILABLE_AT_FORECAST_COLS:
        model_feature_columns.append(col)

    for col in AVAILABLE_AT_FORECAST_COLS:
        model_feature_columns.append(col)
        
    logging.info(f"model_feature_columns: {model_feature_columns}")
    # ===================================================
    # train table specs
    # ===================================================
    activities_table_specs = {
        'bigquery_uri': bq_source_train_uri,
        'table_type': 'FORECASTING_PRIMARY',
        'forecasting_primary_table_metadata': {
            'time_column': TIME_COLUMN,
            'target_column': TARGET_COLUMN,
            'time_series_identifier_column': SERIES_COLUMN,
            'unavailable_at_forecast_columns': UNAVAILABLE_AT_FORECAST_COLS,
            'time_granularity': {
                'unit': data_granularity_unit,
                'quantity': data_granularity_count,
            },
            'predefined_splits_column':PREDEFINED_SPLIT_COL,
        }
    }

    
    return (
        json.dumps(activities_table_specs),  # input_table_specs
        json.dumps(model_feature_columns),  # model_feature_columns

        # json.dumps(COL_TRANSFORMS), # json array
        # json.dumps(UNAVAILABLE_AT_FORECAST_COLS), # json array
        # json.dumps(AVAILABLE_AT_FORECAST_COLS), # json array
        # f'{SERIES_COLUMN}', # string
        # f'{TIME_COLUMN}', # string
        # f'{TARGET_COLUMN}', # string
        # f'{PREDEFINED_SPLIT_COL}', # string
    )

see `AutoMLForecastingTrainingJobRunOp` input data types [here](https://github.com/kubeflow/pipelines/blob/google-cloud-pipeline-components-1.0.41/components/google-cloud/google_cloud_pipeline_components/aiplatform/automl_training_job/automl_forecasting_training_job/component.yaml)

see `ForecastingValidationOp` input [here](https://github.com/kubeflow/pipelines/blob/google-cloud-pipeline-components-1.0.41/components/google-cloud/google_cloud_pipeline_components/experimental/forecasting/validate/component.yaml)

see`ForecastingPrepareDataForTrainOp` input [here](https://github.com/kubeflow/pipelines/blob/google-cloud-pipeline-components-1.0.41/components/google-cloud/google_cloud_pipeline_components/experimental/forecasting/prepare_data_for_train/component.py)

### interpret AutoML metrics

Since we don't have a pre-built-component to access these metrics programmatically, we can use the Vertex AI GAPIC (Google API Compiler), which auto-generates low-level gRPC interfaces to the service.

In [40]:
# For a list of available model metrics, go here:
!gsutil ls gs://google-cloud-aiplatform/schema/modelevaluation/

gs://google-cloud-aiplatform/schema/modelevaluation/classification_metrics_1.0.0.yaml
gs://google-cloud-aiplatform/schema/modelevaluation/forecasting_metrics_1.0.0.yaml
gs://google-cloud-aiplatform/schema/modelevaluation/image_object_detection_metrics_1.0.0.yaml
gs://google-cloud-aiplatform/schema/modelevaluation/regression_metrics_1.0.0.yaml
gs://google-cloud-aiplatform/schema/modelevaluation/text_extraction_metrics_1.0.0.yaml
gs://google-cloud-aiplatform/schema/modelevaluation/text_sentiment_metrics_1.0.0.yaml
gs://google-cloud-aiplatform/schema/modelevaluation/video_action_recognition_metrics_1.0.0.yaml
gs://google-cloud-aiplatform/schema/modelevaluation/video_object_tracking_metrics_1.0.0.yaml


In [50]:
@kfp.v2.dsl.component(
    base_image="python:3.9",
    packages_to_install=[
        "google-cloud-aiplatform==1.22.1",
    ],
)
def interpret_automl_evaluation_metrics(
    region: str, 
    model: Input[Artifact], 
    metrics: Output[Metrics]
):
    """'
    For a list of available forecast metrics, go here: 
        gs://google-cloud-aiplatform/schema/modelevaluation/forecasting_metrics_1.0.0.yaml

    More information on available metrics for different types of models: 
        https://cloud.google.com/vertex-ai/docs/predictions/online-predictions-automl
    """

    import google.cloud.aiplatform.gapic as gapic

    # Get a reference to the Model Service client
    client_options = {"api_endpoint": f"{region}-aiplatform.googleapis.com"}

    model_service_client = gapic.ModelServiceClient(client_options=client_options)

    model_resource_name = model.metadata["resourceName"]

    model_evaluations = model_service_client.list_model_evaluations(
        parent=model_resource_name
    )
    model_evaluation = list(model_evaluations)[0]

    # metrics_dict = {k: [v] for k, v in dict(model_evaluation.metrics).items()}

    available_metrics = [
        'rootMeanSquaredLogError',
        'rSquared',
        'meanAbsoluteError',
        'meanAbsolutePercentageError',
        'rootMeanSquaredError',
        'rootMeanSquaredPercentageError',
        'weightedAbsolutePercentageError',
    ]
    output = dict()
    for x in available_metrics:
        val = model_evaluation.metrics.get(x)
        output[x] = val
        metrics.log_metric(str(x), float(val))

    metrics.log_metric("framework", "AutoML")
    print(output)

### get eval dataset uri

In [51]:
@kfp.v2.dsl.component(base_image='python:3.9')
def get_eval_dataset_path_uri(
    project: str,
    eval_bq_dataset: str,
    model_name: str,
) -> NamedTuple('Outputs',[
    ('model_eval_bigquery_table_uri', str),
    ('eval_bq_dataset', str),
]):
    
    import json
    import logging

    model_1_table_path_name = f'{project}:{eval_bq_dataset}:eval_{model_name}'

    logging.info(model_1_table_path_name)

    return (
        f'bq://{model_1_table_path_name}',
        f'{eval_bq_dataset}',
    )

## Build pipeline

### define vars

In [20]:
VERSION = 'v1'
PIPELINE_TAG = 'train'
BUCKET_NAME = 'vertex-forecast-22'
EXPERIMENT_PREFIX = 'm5-simple-pipe' # custom identifier for organizing experiments

In [21]:
EXPERIMENT_NAME=f'{EXPERIMENT_PREFIX}-{VERSION}'
RUN_NAME = f'run-{time.strftime("%Y%m%d-%H%M%S")}'

GCS_BUCKET_URI =f'gs://{BUCKET_NAME}'
EXPERIMENT_GCS_DIR = f'gs://{BUCKET_NAME}/{EXPERIMENT_NAME}/{RUN_NAME}'

# Stores pipeline executions for each run
PIPELINE_ROOT_PATH = f'{EXPERIMENT_GCS_DIR}/pipeline_root'

print(f"EXPERIMENT_NAME: {EXPERIMENT_NAME}")
print(f"RUN_NAME: {RUN_NAME}")
print(f'EXPERIMENT_GCS_DIR: {EXPERIMENT_GCS_DIR}')
print(f'PIPELINE_ROOT_PATH: {PIPELINE_ROOT_PATH}')

EXPERIMENT_NAME: m5-simple-pipe-v1
RUN_NAME: run-20230329-180343
EXPERIMENT_GCS_DIR: gs://vertex-forecast-22/m5-simple-pipe-v1/run-20230329-180343
PIPELINE_ROOT_PATH: gs://vertex-forecast-22/m5-simple-pipe-v1/run-20230329-180343/pipeline_root


### define pipeline

In [61]:
PIPELINE_NAME = f'{PIPELINE_TAG}-{EXPERIMENT_NAME}'.replace('_', '-')
print(f"PIPELINE_NAME: {PIPELINE_NAME}")

PIPELINE_NAME: train-m5-simple-pipe-v1


In [81]:
@kfp.v2.dsl.pipeline(
  name=PIPELINE_NAME
)
def pipeline(
    vertex_project: str,
    location: str,
    version: str,
    new_bq_dataset: str,
    bq_location: str,
    bq_source_train_uri: str,
    column_spec_gcs_bucket_name: str,
    column_spec_gcs_blob_path:str,
    context_window: int,
    forecast_horizon: int,
    budget_milli_node_hours: int,
    data_granularity_unit: str,
    data_granularity_count: int,
    optimization_objective: str,
    model_name: str,
):
    
    # cretae BQ dataset
    create_bq_dataset_op = (
        create_bq_dataset(
            project=vertex_project,
            new_bq_dataset=new_bq_dataset,
            bq_location=bq_location
        )
    )
    
    # ======================================
    # prep data for forecast train job
    # ======================================
    
    # get column specs
    create_input_table_specs_op = (
        create_input_table_specs(
            project=vertex_project,
            location=location,
            bq_source_train_uri=bq_source_train_uri,
            column_spec_gcs_bucket_name=column_spec_gcs_bucket_name,
            column_spec_gcs_blob_path=column_spec_gcs_blob_path,
            data_granularity_unit=data_granularity_unit,
            data_granularity_count=data_granularity_count,
        )
        .after(create_bq_dataset_op)
    )
    # prebuilt component for forecast data validation
    forecasting_validation_op = (
        gcc_aip_forecasting.ForecastingValidationOp(
            input_tables=str(create_input_table_specs_op.outputs['input_table_specs']),
            validation_theme='FORECASTING_TRAINING',
        )
    )
    
    # prebuilt component for forecast data preprocessing
    forecasting_preprocessing_op = (
      gcc_aip_forecasting.ForecastingPreprocessingOp(
          project=vertex_project,
          input_tables=str(create_input_table_specs_op.outputs['input_table_specs']),
          preprocessing_bigquery_dataset=create_bq_dataset_op.outputs['bq_dataset_name'],
          location=bq_location
      )
    )
    
    # prebuilt component for forecast data prepping
    prepare_data_for_train_op = (
      gcc_aip_forecasting.ForecastingPrepareDataForTrainOp(
          input_tables=(
              str(create_input_table_specs_op.outputs['input_table_specs'])
          ),
          preprocess_metadata=(
              forecasting_preprocessing_op.outputs['preprocess_metadata']
          ),
          model_feature_columns=(
              str(create_input_table_specs_op.outputs['model_feature_columns'])
          )
      )
    )
    
    # pre-built pipeline component for creating Managed dataset (lineage tracking)
    time_series_dataset_create_op = (
      gcc_aip.TimeSeriesDatasetCreateOp(
          display_name=f'train_m5_pipe_{version}',
          bq_source=prepare_data_for_train_op.outputs['preprocess_bq_uri'],
          project=vertex_project,
          location=location,
      )
    )
    
    # get eval BQ destination
    get_eval_dataset_path_uri_op = (
        get_eval_dataset_path_uri(
            project=vertex_project,
            eval_bq_dataset=create_bq_dataset_op.outputs['bq_dataset_name'],
            model_name=model_name,
        )
    )
    
    # ======================================
    # model training with Vertex Forecast
    # ======================================
    
    # vertex forecast automl train job
    model_training_op = (
      gcc_aip.AutoMLForecastingTrainingJobRunOp(
          project=vertex_project,
          location=location,
          display_name=f'train-{model_name}-simple-pipe-{version}',
          model_display_name=f'{model_name}-simple-pipe-{version}',
          dataset=time_series_dataset_create_op.outputs['dataset'],
          optimization_objective=optimization_objective,
          context_window=context_window,
          forecast_horizon=forecast_horizon,
          budget_milli_node_hours=budget_milli_node_hours,
          export_evaluated_data_items=True,
          export_evaluated_data_items_override_destination=True,
          export_evaluated_data_items_bigquery_destination_uri=get_eval_dataset_path_uri_op.outputs['model_eval_bigquery_table_uri'],
          target_column=prepare_data_for_train_op.outputs['target_column'],
          time_column=prepare_data_for_train_op.outputs['time_column'],
          time_series_identifier_column=prepare_data_for_train_op.outputs['time_series_identifier_column'],
          # time_series_attribute_columns=prepare_data_for_train_op.outputs['time_series_attribute_columns'],
          unavailable_at_forecast_columns=prepare_data_for_train_op.outputs['unavailable_at_forecast_columns'],
          available_at_forecast_columns=prepare_data_for_train_op.outputs['available_at_forecast_columns'],
          predefined_split_column_name=prepare_data_for_train_op.outputs['predefined_split_column'],
          #
          data_granularity_unit=data_granularity_unit,
          data_granularity_count=data_granularity_count,
          holiday_regions=['GLOBAL', 'NA', 'US'],
      )
    )
    
    # get trained model
    automl_model = model_training_op.outputs["model"]
    
    # ======================================
    # Vertex Forecast model evaluation
    # ======================================
    
    # Analyzes evaluation AutoML metrics using a custom component.
    automl_eval_op = (
        interpret_automl_evaluation_metrics(
            region=location,
            model=automl_model
        )
    )
    
    automl_eval_metrics = automl_eval_op.outputs["metrics"]

## compile pipeline

In [75]:
PIPELINE_JSON_SPEC_LOCAL = "custom_pipeline_spec.json"

! rm -f $PIPELINE_JSON_SPEC_LOCAL

kfp.v2.compiler.Compiler().compile(
    pipeline_func=pipeline, 
    package_path=PIPELINE_JSON_SPEC_LOCAL,
)

In [76]:
PIPELINES_FILEPATH = f'{PIPELINE_ROOT_PATH}/pipeline_spec.json'
print("PIPELINES_FILEPATH:", PIPELINES_FILEPATH)

!gsutil cp $PIPELINE_JSON_SPEC_LOCAL $PIPELINES_FILEPATH

PIPELINES_FILEPATH: gs://vertex-forecast-22/m5-simple-pipe-v1/run-20230329-180343/pipeline_root/pipeline_spec.json
Copying file://custom_pipeline_spec.json [Content-Type=application/json]...
/ [1 files][ 51.7 KiB/ 51.7 KiB]                                                
Operation completed over 1 objects/51.7 KiB.                                     


## copy artifacts to gcs

> helps with tracking, and accessing files in pipeline steps

In [77]:
LOCAL_COL_SPEC_FILE='column_specs.pkl'

!gsutil cp $LOCAL_COL_SPEC_FILE $EXPERIMENT_GCS_DIR/$LOCAL_COL_SPEC_FILE

Copying file://column_specs.pkl [Content-Type=application/octet-stream]...
/ [1 files][  892.0 B/  892.0 B]                                                
Operation completed over 1 objects/892.0 B.                                      


In [78]:
!gsutil ls $EXPERIMENT_GCS_DIR

gs://vertex-forecast-22/m5-simple-pipe-v1/run-20230329-180343/column_specs.pkl
gs://vertex-forecast-22/m5-simple-pipe-v1/run-20230329-180343/pipeline_root/


# submit pipeline job

In [79]:
from pprint import pprint

# TODO: Service Account address
VERTEX_SA = '934903580331-compute@developer.gserviceaccount.com' 

MODEL_NAME = 'm5-nb7'
NEW_BQ_DATASET = MODEL_NAME.replace("-", "_")

BQ_TRAIN_SOURCE='bq://hybrid-vertex.m5_us.sdk_train_prepped'
COL_SPEC_BQ_BLOB_PATH=f'{EXPERIMENT_NAME}/{RUN_NAME}/{LOCAL_COL_SPEC_FILE}'

# model config
OPTIMIZATION_OBJECTIVE="minimize-rmse"
MILLI_NODE_HRS=1000
# HOLIDAY_REGIONS=['GLOBAL', 'NA', 'US']
FORECAST_GRANULARITY = 'DAY'
DATA_GRANULARITY_COUNT=1
FORECAST_HORIZON = 14
CONTEXT_WINDOW = 14


parameter_values={
    'vertex_project': PROJECT_ID,
    'location': LOCATION,
    'version': VERSION,
    'new_bq_dataset': NEW_BQ_DATASET,
    'bq_location': BQ_LOCATION,
    'bq_source_train_uri': BQ_TRAIN_SOURCE,
    'column_spec_gcs_bucket_name': BUCKET_NAME,
    'column_spec_gcs_blob_path':COL_SPEC_BQ_BLOB_PATH,
    'context_window': CONTEXT_WINDOW,
    'forecast_horizon': FORECAST_HORIZON,
    'budget_milli_node_hours': MILLI_NODE_HRS,
    'data_granularity_unit': FORECAST_GRANULARITY.lower(),
    'data_granularity_count': DATA_GRANULARITY_COUNT,
    'optimization_objective': OPTIMIZATION_OBJECTIVE,
    'model_name': MODEL_NAME,
}
print(f"NEW_BQ_DATASET: {NEW_BQ_DATASET}")
print(f"COL_SPEC_BQ_BLOB_PATH: {COL_SPEC_BQ_BLOB_PATH}\n")
pprint(parameter_values)

NEW_BQ_DATASET: m5_nb7
COL_SPEC_BQ_BLOB_PATH: m5-simple-pipe-v1/run-20230329-180343/column_specs.pkl

{'bq_location': 'US',
 'bq_source_train_uri': 'bq://hybrid-vertex.m5_us.sdk_train_prepped',
 'budget_milli_node_hours': 1000,
 'column_spec_gcs_blob_path': 'm5-simple-pipe-v1/run-20230329-180343/column_specs.pkl',
 'column_spec_gcs_bucket_name': 'vertex-forecast-22',
 'context_window': 14,
 'data_granularity_count': 1,
 'data_granularity_unit': 'day',
 'forecast_horizon': 14,
 'location': 'us-central1',
 'model_name': 'm5-nb7',
 'new_bq_dataset': 'm5_nb7',
 'optimization_objective': 'minimize-rmse',
 'version': 'v1',
 'vertex_project': 'hybrid-vertex'}


In [80]:
job = vertex_ai.PipelineJob(
    display_name=PIPELINE_NAME,
    template_path=PIPELINES_FILEPATH,
    pipeline_root=f'{PIPELINE_ROOT_PATH}',
    failure_policy='fast', # slow | fast
    # enable_caching=False,
    parameter_values=parameter_values,
)

job.run(
    sync=False,
    service_account=VERTEX_SA,
)

Creating PipelineJob
PipelineJob created. Resource name: projects/934903580331/locations/us-central1/pipelineJobs/train-m5-simple-pipe-v1-20230329202744
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/934903580331/locations/us-central1/pipelineJobs/train-m5-simple-pipe-v1-20230329202744')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/train-m5-simple-pipe-v1-20230329202744?project=934903580331
PipelineJob projects/934903580331/locations/us-central1/pipelineJobs/train-m5-simple-pipe-v1-20230329202744 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/934903580331/locations/us-central1/pipelineJobs/train-m5-simple-pipe-v1-20230329202744 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/934903580331/locations/us-central1/pipelineJobs/train-m5-simple-pipe-v1-20230329202744 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/93